In [ ]:
%matplotlib inline

# 神经网络

可以使用 `torch.nn` 包构建神经网络。

现在你已经对 `autograd` 有了一定了解，而 `nn` 是依赖于 `autograd` 来定义模型并区分的。
一个 `nn.Module` 包括 layers 和一个返回 `output` 的方法 `forward(input)` 。

例如下面这个用于分类数字图像的网络：

![convnet](./img/mnist.png)

这是一个简单的前馈网络。它接收输入并馈送到网络中，输入经过一层一层的网络之后最终得到输出。

一个神经网络的典型训练过程如下：

- 定义有一些可学习参数（或权重）的神经网络
- 在输入数据集上进行迭代
- 让输入通过网络
- 计算损失（loss ，其实就是输出与标准值 label 之间的差距）
- 将梯度反向传播给网络参数
- 更新网络权重，通常使用一个简单的更新规则：
  `weight = weight - learning_rate * gradient`

## 定义网络

让我们来定义这个网络吧：


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 个输入图像通道， 6 个输出通道， 5 x 5 见方的卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 一个仿射变换： y = Wx + b
        # 仿射变换，又称仿射映射，是指在几何中，一个向量空间进行一次线性变换并接上一个平移，变换为另一个向量空间。
        # 一个对向量 x 平移 b ， 与旋转放大缩小 W 的仿射变换为 y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 在一个 2 x 2 的窗上进行最大池化
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 如果 size 是和正方形，则只能指定一个数字
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 除批次维度外的所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

你只需要定义 `forward` 函数，然后（计算梯度的） `backward` 函数就会使用 `autograd` 为你自动定义好。
你可以在 `forward` 函数中使用任何 Tensor 的操作。

模型的可学习参数由 `net.parameters()` 返回。


In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1 的权重

让我们用一个随机的 32 x 32 的输入来试一下。
注意：这个网络（LeNet）的预期输入大小是 32 x 32 的。如果你想在 MNIST 数据集上使用这个网络，请把图片大小 resize 到 32 x 32 。


In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

将所有参数的梯度缓存清零并使用随机梯度进行反向传播：


In [ ]:
net.zero_grad()
out.backward(torch.randn(1, 10))

>**注意**  
>`torch.nn` 只支持小批量。整个 `torch.nn` 包只支持小批量样本的输入，并且不能是单个样本。  
>例如 `nn.Conv2d` 在 `nSamples x nChannels x Height x Width` 的 4D Tensor 中执行。  
>如果你只有一个样本，你需要使用 `input.unsqueeze(0)` 来生成一个假的批量维度。

在继续之前，让我们回顾一下之前你见过的所有类。

**回顾：**
- `torch.Tensor` - 一个具有像 `backward()` 这种 autograd 操作支持的*多维数组*。并且存有*关于 tensor 的梯度*。
- `nn.Module` - 神经网络模块。*方便的封装参数的方法*，具有移动到 GPU 、导出、加载等功能。
- `nn.Parameter` - 一种 Tensor ，*当被指派为 `Module` 的属性时，自动注册为 `Module` 的一个参数*。
- `autograd.Function` - 实现了 *autograd 操作的前向和后向的定义*。每一个 `Tensor` 操作创建至少一个 `Function` 节点，该节点连接到创建 `Tensor` 并*编码了它自己历史*的 `Function` 上。

**至此，我们已经讨论过：**
- 定义一个神经网络
- 网络如何处理输入以及调用 backward

**尚未讨论：**
- 计算损失
- 更新网络权重

## 损失函数
损失函数以 (output, target) 参数对作为输入，计算 output 和 target 之间距离的估值。

在 nn 包下有几种不同的[损失函数](https://pytorch.org/docs/nn.html#loss-functions)。
一个简单的损失函数是： `nn.MSELoss` ，它计算 output 和 target 之间的均方差。

下面是例子：


In [ ]:
output = net(input)
target = torch.randn(10)  # 例如一个虚拟的 target
target = target.view(1, -1)  # 使它与 output 形状相同
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

现在如果你使用 `.grad_fn` 属性反向跟踪 `loss` ，你将看到一个如下的计算图：

```
    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss
```

所以，当我们调用 `loss.backward()` 的时候，整个图将根据损失求微分，所有在图中有 `requires_grad=True` 的 Tensor 都将有一个 `.grad` Tensor 累积到梯度上。

为了详细说明，让我们反向走几步：


In [ ]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## 反向传播
想要反向传播误差，我们只需要 `loss.backward()` 。
你需要清理之前存在的梯度，否则新的梯度将累加到之前存在的梯度上。

现在我们将调用 `loss.backward()` ，然后查看 conv1 的 bias 的梯度在 backward 前后的变化。


In [ ]:
net.zero_grad()     # 将所有参数的梯度缓存清零

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

现在我们已经知道如何使用损失函数了。

**稍后阅读：**

　　神经网络软件包包括构建深度神经网络的各种模块和损失函数。[这里](https://pytorch.org/docs/nn)有完整的文档列表。

**还剩最后一步要学：**

- 更新神经网络的权重

## 更新权重
实践中使用的最简单的更新规则是随机梯度下降（SGD）：

`weight = weight - learning_rate * gradient`

我们可以用简单的 Python 代码实现：

```python
    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)
```

然而，当你使用神经网络的时候，你想要使用各种不用的更新规则，比如 SGD 、 Nesterov-SGD 、 Adam 、 RMSProp 等等。
为了使这变为可能，我们构建了一个小的软件包： `torch.optim` 来实现所有这些方法。它的使用非常简单：


In [ ]:
import torch.optim as optim

# 构建你的优化器
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 在你的训练循环过程中：
optimizer.zero_grad()   # 将梯度缓存清零
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # 更新

>注意  
>观察为何梯度缓存必须使用 `optimizer.zero_grad()` 手动设置为 0 。
>这是因为在“反向传播”部分中解释过的：梯度是累加的。
